In [316]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
import time

In [317]:
def read_small():
    data_path = 'ml-latest-small/'
    movies = pd.read_csv(data_path + 'movies.csv')
    ratings = pd.read_csv(data_path + 'ratings.csv')
    return movies, ratings

def read_10M():
    data_path = 'ml-10M100K/'
    movies = pd.read_csv(data_path + 'movies.dat', delimiter='::', engine='python')
    ratings = pd.read_csv(data_path + 'ratings.dat', delimiter='::', engine='python')
    return movies, ratings

In [324]:
movies, ratings = read_small()

In [325]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [326]:
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [327]:
np.unique(ratings.userId.values).shape

(671,)

In [328]:
movie_count = len(movies)
user_count = max(ratings.userId)

mean_score = ratings.rating.values.mean()

In [329]:
all_scores = np.zeros((user_count, max(movies.movieId)))
for (user, movie, rating, _) in ratings.values:
    all_scores[int(user)-1,int(movie)-1] = rating
all_scores = all_scores[:,movies.movieId.values.astype(int)-1]

In [330]:
def train_test_split(scores, seed=1234):
    N = scores.shape[0]
    test_size = N / 10
    np.random.seed(seed)
    test_idx = np.random.choice(N, test_size, replace=False)
    test_set = scores[test_idx]
    train_set = np.delete(scores, test_idx, axis=0)
    return train_set, test_set

def query_target_split(test_set, seed=1234):
    query_set = test_set.copy()
    target_set = test_set.copy()
    for u in xrange(test_set.shape[0]):
        rated = np.where(test_set[u] > 0)[0]
        target = np.random.choice(rated, rated.size / 2, replace=False)
        query_set[u,target] = 0
    target_set[np.where(query_set > 0)] = 0
    return query_set, target_set

def evaluate_ratings(scores, target, method='nmae'):
    mask = target > 0
    abs_dists = np.abs(scores * mask - target)
    if method == 'mae':
        return abs_dists.sum() / mask.sum()
    elif method == 'nmae':
        return abs_dists.sum() / mask.sum() / (5. - .5)
    elif method == 'rmse':
        return np.sqrt((abs_dists**2).sum() / mask.sum())
    else:
        raise Exception('Invalid argument value: method')
        
def compare_genres(userID, recoms):
    genres = {}
    for c in map(lambda x: get_info(x,'genres'), ratings.movieId.values[np.where(ratings.userId.values == userID)]):
        for g in c[0].split('|'):
            if g not in genres:
                genres[g] = 0
            genres[g] += 1
            
    recom_genres = {}
    for c in movies.genres.values[recoms[:100]]:
        for g in c.split('|'):
            if g not in recom_genres:
                recom_genres[g] = 0
            recom_genres[g] += 1
            
    return sorted(genres.items(), key=lambda x: x[1], reverse=True), \
           sorted(recom_genres.items(), key=lambda x: x[1], reverse=True)

In [331]:
train, test = train_test_split(all_scores)
query, target = query_target_split(test)

In [332]:
def get_info(movieId, info='title'):
    return movies[info].values[np.where(movies.movieId.values == movieId)]

def get_user_ratings(userId):
    return ratings.values[np.where(ratings.userId == userId)][:,1:]

In [333]:
def user_baselines(train, query=None, mean_score=None):
    if query is None:
        query = train
    if mean_score is None:
        mean_score = train.sum() / np.count_nonzero(train)
    baselines = query.sum(axis=1) / (query > 0).sum(axis=1) - mean_score
    return np.nan_to_num(baselines)

def item_baselines(train, usr_baselines=None, mean_score=None):
    if mean_score is None:
        mean_score = train.sum() / np.count_nonzero(train)
    if usr_baselines is None:
        usr_baselines = user_baselines(train, mean_score=mean_score)
    nonzeros = (train > 0).sum(axis=0)
    usr_baselines = usr_baselines[:, np.newaxis] * (train > 0)
    preds = (train.sum(axis=0) - usr_baselines.sum(axis=0)) / nonzeros - mean_score
    return np.nan_to_num(preds)

# ta funkcja zwraca szacowania nawet istniejących ocen

def baseline_predictors(train, query=None, mean_score=None):
    if query is None:
        query = train
    if mean_score is None:
        mean_score = train.sum() / np.count_nonzero(train)
    usr_baselines = user_baselines(train, query, mean_score)
    itm_baselines = item_baselines(train, mean_score=mean_score)
    return mean_score + usr_baselines[:, np.newaxis] + itm_baselines

In [334]:
def per_user_means(scores):    
    return scores.sum(axis=1) / (scores > 0).sum(axis=1)

def per_user_stds(scores):
    means = per_user_means(scores)
    nonzeros = (scores > 0).sum(axis=1)
    sums = scores.sum(axis=1)
    sq_sums = (scores**2).sum(axis=1)
    return np.sqrt((sq_sums + nonzeros*means**2 - 2*means*sums) / nonzeros)

# user-user CF

In [335]:
def scores_cf_uu(train, query, n, subtract_mean=True, div_by_std=True, metric='pearson'):
    assert not (div_by_std and not subtract_mean)
    
    train_rated = train > 0
    train_means = per_user_means(train)
    train_stds = per_user_stds(train)
    
    query_rated = query > 0    
    query_means = per_user_means(query)
    query_stds = per_user_stds(query)
    
    query_centered = (query - query_means[:, np.newaxis]) * query_rated
    train_centered = (train - train_means[:, np.newaxis]) * train_rated
    
    if metric == 'cosine':
        dists = cdist(query_centered, train_centered, metric='cosine')
    elif metric == 'pearson':
        numerator = query_centered.dot(train_centered.T)
        denom1 = (query_centered**2).dot(train_rated.T)
        denom2 = query_rated.dot((train_centered**2).T)
        dists = numerator / np.sqrt(denom1 * denom2)

#   dists:          num_query x              num_train
#   train_rated.T:              num_movies x num_train
#   per_item_dists: num_query x num_movies x num_train
#   order:          num_query x num_movies x n

    dists_zero_idx = np.where((dists == 0)[:, np.newaxis] * train_rated.T[np.newaxis])
    
    per_item_dists = dists[:, np.newaxis] * train_rated.T[np.newaxis]
    per_item_dists[per_item_dists == 0] = np.nan
    per_item_dists[dists_zero_idx] = 0

    order = per_item_dists.argpartition(axis=2, kth=n-1)[:, :, :n]
    order_idx = (np.arange(query.shape[0])[:, np.newaxis, np.newaxis],
                 np.arange(query.shape[1])[np.newaxis, :, np.newaxis],
                 order)
    
    weights = per_item_dists[order_idx]
    
    ratings_gathered = (~np.isnan(weights)).sum(axis=2)
    
    weights = np.nan_to_num(weights)
    
    scores = train.T[np.arange(train.shape[1])[:, np.newaxis], order]
    if subtract_mean:
        multiplier = train_stds[order] if div_by_std else 1
        scores = (scores - train_means[order]) / multiplier
        
    estimated_ratings = (scores * weights).sum(axis=2) / (np.abs(weights)).sum(axis=2)
    estimated_ratings[query_rated] = query[query_rated]
    
    # dla ocen nan dajemy baseline predictor
    
    estimated_ratings[np.isnan(estimated_ratings)] = baseline_predictors(train, query)[np.isnan(estimated_ratings)]
    
    if subtract_mean:
        multiplier = query_stds[:, np.newaxis] if div_by_std else 1.        
        estimated_ratings = query_means[:, np.newaxis] + multiplier * estimated_ratings
        
    return estimated_ratings, ratings_gathered

In [336]:
def recommend_cf_uu(userId, n, scores, min_n=0, subtract_mean=True, div_by_std=True, metric='pearson'):
    
    estimated_ratings, ratings_gathered = scores_cf_uu(scores, scores[userId-1][np.newaxis], n, 
                                                       subtract_mean, div_by_std, metric)
    estimated_ratings, ratings_gathered = estimated_ratings[0], ratings_gathered[0]
    estimated_ratings[scores[userId-1] > 0] = np.nan
    estimated_ratings[ratings_gathered < min_n] = np.nan
    
    movie_order = (-estimated_ratings).argsort()
    
    return movie_order + 1, estimated_ratings[movie_order], ratings_gathered[movie_order]

# item-item CF

In [337]:
def calculate_similarities_cf_ii(scores, nan_to_1=True):
    dists = cdist(scores.T, scores.T, metric='cosine')
    if nan_to_1:
        dists[np.isnan(dists)] = 1
    return dists

In [338]:
def scores_cf_ii(train, query, n, dists=None, batch_size=100, verbose=False):
    if dists is None:
        raise NotImplementedError('distances have to be precomputed')
        
    baselines = baseline_predictors(train, query)
    estimated_ratings = []
    
    query_rated = query > 0
        
    idx = 0
    
    while idx < query.shape[1]:
        
        if verbose:
            print "Starting batch {} out of {}".format((idx+1) / batch_size, (query.shape[1] + 1) / batch_size)
        
        batch_idx = slice(idx, idx + batch_size)
        
        batch_query = query[:, batch_idx]        
        batch_query_rated = query_rated[:, batch_idx]

    #   batch_query:    num_query x batch_size
    #   dists:                      num_movies x num_movies
    #   per_user_dists: num_query x batch_size x num_movies
    #   order:          num_query x batch_size x n

        dists_zero_idx = np.where((dists[batch_idx] == 0)[np.newaxis] * query_rated[:, np.newaxis, :])

        per_user_dists = query_rated[:, np.newaxis, :] * dists[np.newaxis, batch_idx]
        per_user_dists[per_user_dists == 0] = np.nan
        per_user_dists[dists_zero_idx] = 0

        order = per_user_dists.argpartition(axis=2, kth=n-1)[:, :, :n]
        order_idx = (np.arange(batch_query.shape[0])[:, np.newaxis, np.newaxis],
                     np.arange(batch_query.shape[1])[np.newaxis, :, np.newaxis],
                     order)    

        weights = np.nan_to_num(per_user_dists[order_idx])

        batch_baselines = baselines[:, batch_idx]

        batch_estimated_ratings = query[np.arange(query.shape[0])[:, np.newaxis, np.newaxis], order] - \
                                  batch_baselines[:, :, np.newaxis]
        batch_estimated_ratings = (batch_estimated_ratings * weights).sum(axis=2) / (np.abs(weights).sum(axis=2)) + \
                                  batch_baselines
        batch_estimated_ratings[batch_query_rated] = batch_query[batch_query_rated]
        
        estimated_ratings.append(batch_estimated_ratings)
        
        idx += batch_size
        
    estimated_ratings = np.hstack(estimated_ratings)
    
    return estimated_ratings

In [339]:
def recommend_cf_ii(userId, n, scores, dists=None):
    if dists is None:
        raise NotImplementedError('distances have to be precomputed')

    user_scores = scores[userId-1]
    user_rated = user_scores > 0

    dists = dists[:, user_rated]
    if dists.shape[1] > n:
        order = dists.argpartition(axis=1, kth=n-1)[:, :n]
    else:
        order = np.indices((movie_count, dists.shape[1]))[1]
    weights = dists[np.arange(movie_count)[:, np.newaxis], order]
    
    base_user_preds = baseline_predictors(scores)[userId]
    estimated_ratings = user_scores[user_rated][order] - base_user_preds[:, np.newaxis]
    estimated_ratings = (estimated_ratings * weights).sum(axis=1) / \
                        (np.abs(weights)).sum(axis=1) + base_user_preds
    estimated_ratings[user_rated] = np.nan
    
    movie_order = (-estimated_ratings).argsort()
    
    return movie_order + 1, estimated_ratings[movie_order]

In [340]:
def recommend_cf_ii(userId, n, scores, dists=None):
    if dists is None:
        raise NotImplementedError('distances have to be precomputed')

    estimated_ratings = scores_cf_ii(scores, scores[userId-1][np.newaxis], n, dists=dists, 
                                     batch_size=scores.shape[1])[0]
    estimated_ratings[scores[userId-1] > 0] = np.nan
    
    movie_order = (-estimated_ratings).argsort()
    
    return movie_order + 1, estimated_ratings[movie_order]

# baseline evaluation

In [341]:
evaluate_ratings(baseline_predictors(train, query), target, method='mae')

0.70082927125178218

# testy uu

In [342]:
means = per_user_means(all_scores)
stds = per_user_stds(all_scores)

In [343]:
userId = 55

t0 = time.time()
recoms, scores, sample_size = recommend_cf_uu(userId, 50, all_scores, 10, metric='cosine')
print time.time()-t0
scores[:10]

0.330183029175


array([ 4.74451171,  4.73295588,  4.60888204,  4.50147045,  4.47123194,
        4.44896116,  4.43687627,  4.43687572,  4.42654412,  4.40487016])

In [269]:
S = scores_cf_uu(train, query, 50, metric='cosine')[0]

In [270]:
evaluate_ratings(S, target, method='mae')

0.74541410454700974

# testy ii

In [344]:
dists = calculate_similarities_cf_ii(all_scores)

In [345]:
userId = 55

t0 = time.time()
recoms, scores = recommend_cf_ii(userId, 50, all_scores, dists=dists)
print time.time()-t0
scores[:10]

1.00487589836


array([ 3.67693713,  3.67064765,  3.66765763,  3.66679177,  3.66579527,
        3.66579527,  3.66579527,  3.6656915 ,  3.66465039,  3.66430297])

In [163]:
dists = calculate_similarities_cf_ii(train)

In [307]:
S_ii = scores_cf_ii(train, query, 40, dists, batch_size=500)

In [313]:
evaluate_ratings(S_ii, target, method='mae')

0.70224992060399649

In [ ]:
# TODO
# czemu baseline ma najlepszy wynik?
# slope one
# przerobić scores tak, żeby robiło argpartition na mniejszych tablicach